In [1]:
# 필요한 라이브러리 설치
!pip install -q kiwipiepy  #한국어 형태소 분석
!pip install -q wget       #HTTP, HTTPS 프로토콜을 사용하여 파일을 다운
!pip install -q sgmllib3k  # Python 2의 sgmllib를 Python 3에서 사용할 수 있게 해주는 라이브러리, SGML 파싱지원

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [2]:
import base64
import requests
from IPython.display import Image, display
import os
import time
from openai import OpenAI
from openai import AssistantEventHandler
from typing_extensions import override
from google.colab import userdata

# Open AI API Key 설정
openai_api_key = userdata.get('OPENAI_API_KEY')

In [3]:
class MultiModal:
    def __init__(self, model, system_prompt=None, user_prompt=None):
        self.model = model
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
        self.init_prompt()

    def init_prompt(self):
        if self.system_prompt is None:
            self.system_prompt = "You are a helpful assistant who helps users to write a report related to images in Korean."
        if self.user_prompt is None:
            self.user_prompt = "Explain the images as an alternative text in Korean."

    # 이미지를 base64로 인코딩하는 함수 (URL)
    def encode_image_from_url(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            image_content = response.content
            if url.lower().endswith((".jpg", ".jpeg")):
                mime_type = "image/jpeg"
            elif url.lower().endswith(".png"):
                mime_type = "image/png"
            else:
                mime_type = "image/unknown"
            return f"data:{mime_type};base64,{base64.b64encode(image_content).decode('utf-8')}"
        else:
            raise Exception("Failed to download image")

    # 이미지를 base64로 인코딩하는 함수 (파일)
    def encode_image_from_file(self, file_path):
        with open(file_path, "rb") as image_file:
            image_content = image_file.read()
            file_ext = os.path.splitext(file_path)[1].lower()
            if file_ext in [".jpg", ".jpeg"]:
                mime_type = "image/jpeg"
            elif file_ext == ".png":
                mime_type = "image/png"
            else:
                mime_type = "image/unknown"
            return f"data:{mime_type};base64,{base64.b64encode(image_content).decode('utf-8')}"

    # 이미지 경로에 따라 적절한 함수를 호출하는 함수
    def encode_image(self, image_path):
        if image_path.startswith("http://") or image_path.startswith("https://"):
            return self.encode_image_from_url(image_path)
        else:
            return self.encode_image_from_file(image_path)

    def display_image(self, encoded_image):
        display(Image(url=encoded_image))

    def create_messages(
        self, image_url, system_prompt=None, user_prompt=None, display_image=True
    ):
        encoded_image = self.encode_image(image_url)
        if display_image:
            self.display_image(encoded_image)

        system_prompt = (
            system_prompt if system_prompt is not None else self.system_prompt
        )

        user_prompt = user_prompt if user_prompt is not None else self.user_prompt

        # 인코딩된 이미지를 사용하여 다른 처리를 수행할 수 있습니다.
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"{encoded_image}"},
                    },
                ],
            },
        ]
        return messages

    def invoke(
        self, image_url, system_prompt=None, user_prompt=None, display_image=True
    ):
        messages = self.create_messages(
            image_url, system_prompt, user_prompt, display_image
        )
        response = self.model.invoke(messages)
        return response.content

    def batch(
        self,
        image_urls: list[str],
        system_prompts: list[str] = [],
        user_prompts: list[str] = [],
        display_image=False,
    ):
        messages = []
        for image_url, system_prompt, user_prompt in zip(
            image_urls, system_prompts, user_prompts
        ):
            message = self.create_messages(
                image_url, system_prompt, user_prompt, display_image
            )
            messages.append(message)
        response = self.model.batch(messages)
        return [r.content for r in response]

    def stream(
        self, image_url, system_prompt=None, user_prompt=None, display_image=True
    ):
        messages = self.create_messages(
            image_url, system_prompt, user_prompt, display_image
        )
        response = self.model.stream(messages)
        return response


class OpenAIStreamHandler(AssistantEventHandler):
    @override
    def on_text_delta(self, delta, snapshot):
        return delta.value


class OpenAIAssistant:
    """
    OpenAI 어시스턴트를 관리하는 클래스입니다.
    이 클래스는 OpenAI API를 사용하여 파일 업로드, 어시스턴트 생성, 대화 관리 등의 기능을 제공합니다.
    """

    def __init__(self, configs):
        """
        OpenAIAssistant 클래스의 생성자입니다.

        :param configs: 설정 정보를 담은 딕셔너리
        configs = {
            "OPENAI_API_KEY": "OPENAI_API_KEY",
            "instructions": "사용자 입력 RAG 프롬프트미 설정시 기본 값",
            "PROJECT_NAME": "PDF-INTERVIEW-RAG-TEST", # 프로젝트 이름
            "model_name": "gpt-4o", # openai 모델 이름
            "chunk_size": 1000, # 청크 크기
            "chunk_overlap": 100, # 청크 중복 크기
        }
        """
        self.client = OpenAI(api_key=configs["OPENAI_API_KEY"])
        self.model = configs.get("model_name", "gpt-4o")
        self.instructions = configs.get("instructions", "")
        self.project_name = configs.get("PROJECT_NAME", "PDF-INTERVIEW-RAG-TEST")
        self.chunk_size = configs.get("chunk_size", 800)
        self.chunk_overlap = configs.get("chunk_overlap", 400)

        self.messages = []
        self.thread_id = None

    def upload_file(self, filepath):
        """
        파일을 OpenAI 서버에 업로드합니다.

        :param filepath: 업로드할 파일의 경로
        :return: 업로드된 파일의 ID
        """
        file = self.client.files.create(file=open(filepath, "rb"), purpose="assistants")
        return file.id

    def create_new_assistant(self, file_ids):
        """
        새로운 어시스턴트를 생성합니다.

        :param file_ids: 어시스턴트에 연결할 파일 ID 리스트
        :return: 생성된 어시스턴트의 ID와 벡터 스토어의 ID
        """
        # 현재 사용 사례에는 파일 검색 도구만 관련이 있습니다
        tools = [{"type": "file_search"}]

        chunking_strategy = {
            "type": "static",
            "static": {
                "max_chunk_size_tokens": self.chunk_size,
                "chunk_overlap_tokens": self.chunk_overlap,
            },
        }

        # 벡터 스토어 생성
        vector_store = self.client.beta.vector_stores.create(
            name=self.project_name,
            file_ids=file_ids,
            chunking_strategy=chunking_strategy,
        )
        tool_resources = {"file_search": {"vector_store_ids": [vector_store.id]}}

        # 어시스턴트 생성
        assistant = self.client.beta.assistants.create(
            name=self.project_name,
            instructions=self.instructions,
            model=self.model,
            tools=tools,
            tool_resources=tool_resources,
        )
        assistant_id = assistant.id
        vector_id = vector_store.id
        return assistant_id, vector_id

    def setup_assistant(self, assistant_id):
        """
        어시스턴트 ID를 설정합니다.

        :param assistant_id: 설정할 어시스턴트 ID
        """
        self.assistant_id = assistant_id

    def setup_vectorstore(self, vector_id):
        """
        벡터 스토어 ID를 설정합니다.

        :param vector_id: 설정할 벡터 스토어 ID
        """
        self.vector_id = vector_id

    def _start_assistant_thread(self, prompt):
        """
        어시스턴트와의 대화 스레드를 시작합니다.

        :param prompt: 초기 프롬프트 메시지
        :return: 생성된 스레드의 ID
        """
        # 메시지 초기화
        self.messages = [{"role": "user", "content": prompt}]

        # 스레드 생성
        tool_resources = {"file_search": {"vector_store_ids": [self.vector_id]}}
        thread = self.client.beta.threads.create(
            messages=self.messages, tool_resources=tool_resources
        )

        return thread.id

    def _run_assistant(self, thread_id):
        """
        어시스턴트를 실행합니다.

        :param thread_id: 실행할 스레드의 ID
        :return: 실행된 작업의 ID
        """
        run = self.client.beta.threads.runs.create(
            thread_id=thread_id, assistant_id=self.assistant_id
        )
        return run.id

    def _check_run_status(self, thread_id, run_id):
        """
        실행 상태를 확인합니다.

        :param thread_id: 스레드 ID
        :param run_id: 실행 ID
        :return: 실행 상태
        """
        run = self.client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return run.status

    def _retrieve_thread_messages(self, thread_id):
        """
        스레드의 메시지를 검색합니다.

        :param thread_id: 검색할 스레드의 ID
        :return: 메시지 리스트
        """
        thread_messages = self.client.beta.threads.messages.list(thread_id)
        list_messages = thread_messages.data
        thread_messages = []
        for message in list_messages:
            obj = {}
            obj["content"] = message.content[0].text.value
            obj["role"] = message.role
            thread_messages.append(obj)
        return thread_messages[::-1]

    def _add_messages_to_thread(self, thread_id, user_message):
        """
        스레드에 새 메시지를 추가합니다.

        :param thread_id: 메시지를 추가할 스레드의 ID
        :param user_message: 추가할 사용자 메시지
        :return: 추가된 메시지 객체
        """
        thread_message = self.client.beta.threads.messages.create(
            thread_id, role="user", content=user_message
        )
        return thread_message

    def invoke(self, message):
        """
        어시스턴트에게 메시지를 보내고 응답을 받습니다.

        :param message: 보낼 메시지
        :return: 어시스턴트의 응답
        """
        if len(self.messages) == 0:
            self.thread_id = self._start_assistant_thread(message)
        else:
            self._add_messages_to_thread(self.thread_id, message)

        run_id = self._run_assistant(self.thread_id)
        while self._check_run_status(self.thread_id, run_id) != "completed":
            time.sleep(1)
        answer = self._retrieve_thread_messages(self.thread_id)
        return answer[-1]["content"]

    def stream(self, message):
        """
        어시스턴트에게 메시지를 보내고 응답을 스트림으로 받습니다.

        :param message: 보낼 메시지
        :return: 어시스턴트의 응답 스트림
        """
        if len(self.messages) == 0:
            self.thread_id = self._start_assistant_thread(message)
        else:
            self._add_messages_to_thread(self.thread_id, message)

        handler = OpenAIStreamHandler()

        with self.client.beta.threads.runs.stream(
            thread_id=self.thread_id,
            assistant_id=self.assistant_id,
            instructions=self.instructions,
            event_handler=handler,
        ) as stream:
            for text in stream.text_deltas:
                yield text

    def list_chat_history(self):
        """
        대화 기록을 반환합니다.

        :return: 대화 기록 리스트
        """
        return self._retrieve_thread_messages(self.thread_id)

    def clear_chat_history(self):
        """
        대화 기록을 초기화합니다.
        """
        self.messages = []
        self.thread_id = None

In [4]:
# RAG 시스템 프롬프트 입력
_DEFAULT_RAG_INSTRUCTIONS = """업로드된 자기소개서의 내용을 기반으로 엄격한 압박 면접을 진행하는 면접관 역할을 맡아주세요. 실제 면접처럼 한가지의 질문 혹은 요청을 합니다.
첫시작은 안녕하세요 000지원자씨 면접 시작하겠습니다. 먼저 간단하게 자기소개 부탁드립니다. 라고 시작해.
내가 답변을 하면 나의 답변이 업로드한 자기소개서 pdf 파일 내용과 일치하는지 확인하고, 면접 과정에서 모호하거나 구체적인 설명이 부족한 부분이 있다면 추가 질문을 해주세요. 이때도 한가지만 질문합니다. 또한, 자기소개서에 드러나지 않은 내 전공 분야나 관련된 지식에 대해서도 깊은 이해를 평가할 수 있는 질문을 해주세요.

# Steps

1. **자기소개서 분석**: 내가 업로드한 자기소개서 내용을 바탕으로 주요 이슈나 질문 거리가 될 만한 부분을 도출하세요.
2. **답변 검증 및 질문**: 내가 면접 과정에서 한 답변이 자기소개서의 내용과 일치하는지 확인하고, 모호한 부분이 있다면 다시 질문하거나 이유를 물어봐 주세요.
3. **깊이 있는 추가 질문**: 내 분야와 관련된 지식에 대한 질문을 통해 나의 이해도를 검증하는 질문을 해주세요.
4. **압박 면접 진행**: 편안함을 허용하지 않는 방향으로, 내가 준비되지 않았을 것으로 보일만한 질문이나 예상치 못한 질문을 통해 차분하지만 압박적인 면접을 진행하세요.

# Output Format

다음과 같은 형식으로 진행해주세요:
- 질문 형태로 나에게 한가지 질문을 먼저 해주세요.
- 내가 대답을 하면, 그 대답이 자기소개서와 일치하는지 여부와 함께 추가로 궁금한 점을 지적하거나 다른 질문을 이어가주세요.
- 질문과 검토가 모두 이루어진 후, 결론적으로 나의 답변 평가 또는 조언도 짧게 제공해주세요.  반드시 면접 대화 느낌이 나게 한번에 하나씩 질문해야함 Examples 형식에 맞게 질문해


# Notes

- 가능한 한 압박적인 질문을 지속하며 진정성 있는 답변을 유도해주세요.
- 답변의 모호함이나 어느 정도 준비되지 않은 부분에 대해 의도적으로 도전적인 질문을 해주세요.
- 나의 분야에 대해 심도 있는 질문을 던질 때에는 자기소개서의 내용을 기반으로 파생된 점에 초점을 맞춰주세요.
- 한국어로 질문하세요.
- 서로 상호작용을 하며 면접이 이루어져야합니다. 한번 말할 때 한가지의 질문을 합니다."""


# 설정(configs)
configs = {
    "OPENAI_API_KEY": openai_api_key,  # OpenAI API 키
    "instructions": _DEFAULT_RAG_INSTRUCTIONS,  # RAG 시스템 프롬프트
    "PROJECT_NAME": "INTERVIEW-TEST",  # 프로젝트 이름(자유롭게 설정)
    "model_name": "gpt-4o",  # 사용할 OpenAI 모델 이름(gpt-4o, gpt-4o-mini, ...)
    "chunk_size": 1000,  # 청크 크기
    "chunk_overlap": 100,  # 청크 중복 크기
}


# 인스턴스 생성
assistant = OpenAIAssistant(configs)

In [5]:
# 업로드할 파일 경로
data = "자기소개서_샘플.pdf"

# 파일 업로드 후 file_id 는 잘 보관해 두세요. (대시보드에서 나중에 확인 가능)
file_id = assistant.upload_file(data)

In [6]:
# 업로드한 파일의 ID 리스트 생성
file_ids = [file_id]

# 새로운 어시스턴트 생성 및 ID 받기
assistant_id, vector_id = assistant.create_new_assistant(file_ids)

# 어시스턴트 설정
assistant.setup_assistant(assistant_id)

# 벡터 스토어 설정
assistant.setup_vectorstore(vector_id)


In [7]:
##이미 있는 assistant_id, vector_id 가 있으면 아래 코드로 실행
# assistant_id = "asst_~~~~........"
# vector_id = "vs_~~~~........"

# # 어시스턴트 설정
# assistant.setup_assistant(assistant_id)

# # 벡터 스토어 설정
# assistant.setup_vectorstore(vector_id)

In [8]:
#대화 stream(),invoke() 중 stream()으로 구
for token in assistant.stream("안녕하세요. 지원자 정현정입니다."):
    print(token, end="", flush=True)

안녕하세요 정현정 지원자씨, 면접 시작하겠습니다. 먼저 간단하게 자기소개 부탁드립니다.

In [9]:
for token in assistant.stream("엔지니어 직무에 지원하게 된 동기는 아무나 할 수 없는 특수성 때문입니다. 군 복무 중 정전이 발생한 적이 있습니다. 당직근무를 서고 있던 저는 전기 군무원님과 함께 부대 내의 수변전실로 들어갔습니다. 비상발전기, LBS, VCB 등 다양한 전기설비를 능숙하게 다루어 정전을 해결하는 것을 보고 전기 엔지니어의 꿈을 가지게 되었습니다.입사 후 목표는 디지털 전기 엔지니어가 되는 것입니다."):
    print(token, end="", flush=True)

정현정 지원자님의 자기소개는 자기소개서와 일치합니다. 다음 질문 드리겠습니다.

사물인터넷(IoT) 기술에 대해 관심을 갖게 되었다고 하셨는데, IoT 기술을 활용한 프로젝트에서 가장 큰 도전과제는 무엇이었으며, 어떻게 극복하셨는지 구체적으로 설명해 주시겠습니까?

In [10]:
for token in assistant.stream("프로그래밍 언어에 관한 관심으로 C언어, Java, 파이썬 과목을 수강하여 알고리즘 지식을 쌓았습니다. 이를 바탕으로 사물인터넷(IOT) 기술에 관심을 갖게 되었고 2019 캡스톤 디 자인 경진대회에 도전하는 계기가 되었습니다. 라즈베리파이에서 파이썬 언어를 통해 주 인이 부재중일 때 반려동물에게 먹이를 줄 수 있는 '애니멀 피더' 작품을 구상했고 12개 의 팀이 참가한 대회에서 2등으로 우수상을 받았습니다."):
    print(token, end="", flush=True)

정현정 지원자님의 답변은 자기소개서와 잘 일치합니다. '애니멀 피더' 프로젝트를 진행하며 점퍼선 케이블이 빠져 발생한 문제를 해결하고 추가 기능을 구현하는 과정에서 어려움을 극복한 경험이 잘 드러나 있습니다【12:1†자기소개서_샘플.pdf】.

다음 질문은 더 깊이 있는 전문성을 확인해보기 위해 드리겠습니다. 전공하신 전기공학에서 전력 시스템의 디지털화를 위한 한 가지 혁신 기술 또는 접근 방식에 대해 설명해주시고, 그것이 산업에 미치는 영향을 예측해보시겠습니까?

In [11]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '안녕하세요. 지원자 정현정입니다.', 'role': 'user'},
 {'content': '안녕하세요 정현정 지원자씨, 면접 시작하겠습니다. 먼저 간단하게 자기소개 부탁드립니다.',
  'role': 'assistant'},
 {'content': '엔지니어 직무에 지원하게 된 동기는 아무나 할 수 없는 특수성 때문입니다. 군 복무 중 정전이 발생한 적이 있습니다. 당직근무를 서고 있던 저는 전기 군무원님과 함께 부대 내의 수변전실로 들어갔습니다. 비상발전기, LBS, VCB 등 다양한 전기설비를 능숙하게 다루어 정전을 해결하는 것을 보고 전기 엔지니어의 꿈을 가지게 되었습니다.입사 후 목표는 디지털 전기 엔지니어가 되는 것입니다.',
  'role': 'user'},
 {'content': '정현정 지원자님의 자기소개는 자기소개서와 일치합니다. 다음 질문 드리겠습니다.\n\n사물인터넷(IoT) 기술에 대해 관심을 갖게 되었다고 하셨는데, IoT 기술을 활용한 프로젝트에서 가장 큰 도전과제는 무엇이었으며, 어떻게 극복하셨는지 구체적으로 설명해 주시겠습니까?',
  'role': 'assistant'},
 {'content': "프로그래밍 언어에 관한 관심으로 C언어, Java, 파이썬 과목을 수강하여 알고리즘 지식을 쌓았습니다. 이를 바탕으로 사물인터넷(IOT) 기술에 관심을 갖게 되었고 2019 캡스톤 디 자인 경진대회에 도전하는 계기가 되었습니다. 라즈베리파이에서 파이썬 언어를 통해 주 인이 부재중일 때 반려동물에게 먹이를 줄 수 있는 '애니멀 피더' 작품을 구상했고 12개 의 팀이 참가한 대회에서 2등으로 우수상을 받았습니다.",
  'role': 'user'},
 {'content': "정현정 지원자님의 답변은 자기소개서와 잘 일치합니다. '애니멀 피더' 프로젝트를 진행하며 점퍼선 케이블이 빠져 발생한 문제를 해결하고 추가 기능을 구현하는 과정에서 어려움을 극복한 경험이 잘 드러나 있습니다【12:1†자기소개서_샘플.pdf】.\

In [ ]:
# 대화 초기화
assistant.clear_chat_history()